# 12. 이미지 변형(원근)

## 사다리꼴 이미지 펼치기

In [4]:
import cv2
import numpy as np

img = cv2.imread('newspaper.jpg')

width, height = 640, 240 # 가로 크기 640, 세로 크기 240 으로 결과물 출력

# 네개의 점을 어레이로(그림판에서 각 귀퉁이 좌표 확인)
src = np.array([[511, 352], [1008, 345], [1122, 584], [455, 594]], dtype=np.float32) # input 4개 지점
dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32) # output 4개 지점
# 좌상, 우상, 우하, 좌하 (시계 방향으로 4지점)


matrix = cv2.getPerspectiveTransform(src, dst) # 변환에 적용될 행렬(Matrix) 얻어옴
result = cv2.warpPerspective(img, matrix, (width, height)) # 행렬(Matrix) 대로 변환을 함

cv2.imshow('img', img)
cv2.imshow('result', result)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## 회전된 이미지 올바로 세우기

In [6]:
import cv2
import numpy as np

img = cv2.imread('poker.jpg')

width, height = 530, 710 # 가로 크기 530, 세로 크기 710 으로 결과물 출력

# 네개의 점을 어레이로(그림판에서 각 귀퉁이 좌표 확인)
src = np.array([[702, 143], [1133, 414], [726, 1007], [276, 700]], dtype=np.float32) # input 4개 지점
dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32) # output 4개 지점
# 좌상, 우상, 우하, 좌하 (시계 방향으로 4지점)


matrix = cv2.getPerspectiveTransform(src, dst) # 변환에 적용될 행렬(Matrix) 얻어옴
result = cv2.warpPerspective(img, matrix, (width, height)) # 행렬(Matrix) 대로 변환을 함

cv2.imshow('img', img)
cv2.imshow('result', result)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

## 미니 프로젝트 : 반자동 문서 스캐너

이미지 상에서 마우스 클릭만으로 실시간 찾아지는 영역으로 사각형으로 만드는 작업

### 마우스 이벤트 등록

In [13]:
import cv2

def mouse_handler(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN: # 마우스 왼쪽 버튼 다운
        print('왼쪽버튼 다운')
        print(x, y)
    elif event == cv2.EVENT_LBUTTONUP: # 마우스 왼쪽 버튼 업
        print('왼쪽버튼 업')
        print(x, y)
#     elif event == cv2.EVENT_LBUTTONDBLCLK:
#         print('왼쪽버튼 더블클릭')
#     elif event == cv2.EVENT_MOUSEMOVE:
#         print('마우스 이동')

img = cv2.imread('poker.jpg')
cv2.namedWindow('img') # img라는 이름의 윈도우를 먼저 만들어 두는 것, 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용
cv2.setMouseCallback('img', mouse_handler)
cv2.imshow('img', img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

왼쪽버튼 다운
705 157
왼쪽버튼 업
705 157
왼쪽버튼 다운
1128 422
왼쪽버튼 업
1128 422
왼쪽버튼 다운
270 692
왼쪽버튼 업
275 692
왼쪽버튼 다운
722 1013
왼쪽버튼 업
722 1013
왼쪽버튼 다운
722 1006
왼쪽버튼 업
722 1006


-1

### 프로젝트

In [21]:
import cv2
import numpy as np

src_img = cv2.imread('poker.jpg')

COLOR = (255, 0, 255)
THICKNESS = 3
drawing = False # 선을 그릴지 여부(처음에는 False → 클릭후 부터 True)

point_list = []
def mouse_handler(event, x, y, flags, param):
    global drawing
    dst_img = src_img.copy() # 핸들러안에서 하는 모든 작업을 dst_img라는 이미지에서 작업하는것으로 설정
    
    if event == cv2.EVENT_LBUTTONDOWN: # 마우스 왼쪽 버튼 다운
        print('왼쪽버튼 다운')
        print(x, y)
        drawing = True # 선을 그리기 시작
        point_list.append((x, y))
        
    if drawing:
        prev_point = None # 직선의 시작점
        for point in point_list:
            cv2.circle(dst_img, point, 15, COLOR, cv2.FILLED)
            if prev_point: # 직전의 점이 있다면
                cv2.line(dst_img, prev_point, point, COLOR, THICKNESS, cv2.LINE_AA)
            prev_point = point # 직전의 점을 업데이트
            
        next_point = (x, y) # 핸들러에서 받아온 x, y (계속 갱신되고 있음)
        if len(point_list) == 4:
            show_result() # 결과 출력하는 함수(4번째 점선택하면 선을 긋기전에 결과를 보여주게 됨)
            next_point = point_list[0] # 첫번째 클릭한 점으로
        cv2.line(dst_img, prev_point, next_point, COLOR, THICKNESS, cv2.LINE_AA)
    
    cv2.imshow('img', dst_img)

def show_result():
    width, height = 530, 710 # 가로 크기 530, 세로 크기 710 으로 결과물 출력
    # 네개의 점을 어레이로(그림판에서 각 귀퉁이 좌표 확인)
    src = np.float32(point_list) # input 4개 지점
    dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32) # output 4개 지점
    # 좌상, 우상, 우하, 좌하 (시계 방향으로 4지점)

    matrix = cv2.getPerspectiveTransform(src, dst) # 변환에 적용될 행렬(Matrix) 얻어옴
    result = cv2.warpPerspective(src_img, matrix, (width, height)) # 행렬(Matrix) 대로 변환을 함
    cv2.imshow('result', result)
    
cv2.namedWindow('img') # img라는 이름의 윈도우를 먼저 만들어 두는 것, 여기에 마우스 이벤트를 처리하기 위한 핸들러 적용, img명칭은 자유롭게가능
cv2.setMouseCallback('img', mouse_handler) # 상기의 윈도우 이름을 그대로 사용

cv2.imshow('img', src_img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

왼쪽버튼 다운
699 140
왼쪽버튼 다운
1136 419
왼쪽버튼 다운
721 1003
왼쪽버튼 다운
276 696


-1